In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import prophet
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json
import datetime
import random

In [156]:
dataset_path = '../data/'

pollen_test = pd.read_csv(dataset_path + 'pollen_test.csv')
pollen_train = pd.read_csv(dataset_path + 'pollen_train.csv')
pollen_type = pd.read_csv(dataset_path + 'pollen-type.csv')
submission_example = pd.read_csv(dataset_path + 'submission_example.csv')
weather_data = pd.read_csv(dataset_path + 'weather_data.csv')

In [157]:
with open('../results/prophet.csv', 'w') as f:
    f.write('batch_id,1 day prediction,2 days prediction,3 days prediction\n')

In [158]:
print(max(pollen_test.AMBROSIA))

458


In [159]:
pollen_test.head()

,Unnamed: 0,location,date,ACER,ALNUS,AMBROSIA,ARTEMISIA,BETULA,CANNABACEAE,CARPINUS,...,PLATANUS,POACEAE,POPULUS,QUERCUS,RUMEX,SALIX,TILIA,ULMACEAE,URTICACEAE,batch_id
0,0,БЕОГРАД - НОВИ БЕОГРАД,2022-02-14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,БЕОГРАД - НОВИ БЕОГРАД,2022-02-15,0,5,0,0,6,0,0,...,0,0,6,0,0,0,0,13,0,1
2,2,БЕОГРАД - НОВИ БЕОГРАД,2022-02-16,0,0,0,0,1,0,0,...,0,0,6,0,0,0,0,13,0,1
3,3,БЕОГРАД - НОВИ БЕОГРАД,2022-02-17,0,6,0,0,11,0,0,...,0,0,22,0,0,0,0,30,0,1
4,4,БЕОГРАД - НОВИ БЕОГРАД,2022-02-18,0,12,0,0,2,0,0,...,0,0,47,0,0,0,0,58,0,1


In [160]:
pollen_df = pollen_test.copy()

pollen_df['date'] = pd.to_datetime(pollen_df['date'])

# change the column names
pollen_df['ds'] = pollen_df['date']
pollen_df['y'] = pollen_df['AMBROSIA']
pollen_df['floor'] = 0
pollen_df['cap'] = 2500

pollen_df.drop(['location', 'Unnamed: 0', 'date', 'AMBROSIA'], axis=1, inplace=True)

pollen_df.tail()

,ACER,ALNUS,ARTEMISIA,BETULA,CANNABACEAE,CARPINUS,CELTIS,CHENOP/AMAR.,CORYLUS,CUPRESS/TAXA.,...,RUMEX,SALIX,TILIA,ULMACEAE,URTICACEAE,batch_id,ds,y,floor,cap
1115,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,2,112,2022-10-15,4,0,2500
1116,0,0,1,0,1,0,0,2,0,0,...,0,0,0,0,0,112,2022-10-16,3,0,2500
1117,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-17,2,0,2500
1118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-18,7,0,2500
1119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-19,2,0,2500


In [161]:
def next_three_days(x):
    return [x + datetime.timedelta(days=1), x + datetime.timedelta(days=2), x + datetime.timedelta(days=3)]

def predict_for_dates(m, dates, ratio):
    future = pd.DataFrame({'ds': dates})
    future['floor'] = 0
    future['cap'] = 2500
    
    forecast = m.predict(future)
    ans = []
    for forecast_row in forecast.itertuples():
        val = int(ratio * random.randrange(int(forecast_row.yhat_lower), int(forecast_row.yhat_upper)))
        val = max(0, val)
        ans.append(val)

        '''
        if forecast_row.yhat_lower < 0:
            ans.append(0)
        else:
            # pick random value between yhat_lower and yhat_upper
            # ans.append(random.randrange(int(forecast_row.yhat_lower), int(forecast_row.yhat_upper)))
            val = int(ratio * random.randrange(int(forecast_row.yhat_lower), int(forecast_row.yhat_upper)))
            print(val, ratio, forecast_row.yhat, forecast_row.yhat_lower, forecast_row.yhat_upper)
            ans.append(val)
        '''

    return ans

## Start Test

In [162]:
BATCH_SIZE = 113
for batch_id in range(1, BATCH_SIZE):
    print('--------- Batch id: ' + str(batch_id) + ' ---------')

    # extract pollen_df where column batch_id == batch_id
    test_df = pollen_df[pollen_df['batch_id'] == batch_id]
    # Drop index for test_df
    test_df.reset_index(inplace=True, drop=True)

    with open('../models/prophet.json', 'r') as fin:
        m = model_from_json(fin.read())  # Load model

    forecast = m.predict(test_df)
    
    # Make a list of difference between yhat and y
    ratio = np.mean([test_df['y'][i] / max(1e-9, forecast['yhat'][i]) for i in range(len(forecast))])
    print(ratio)

    # 3 days after test_df
    last_date = test_df['ds'].iloc[-1]
    predict_dates = next_three_days(last_date)

    results = predict_for_dates(m, predict_dates, ratio)

    print(last_date, list(test_df['y']))
    print([int(ratio * x) for x in forecast['yhat']], results)

    # Save results to result.csv
    with open('../results/prophet.csv', 'a') as fout:
        results = str(batch_id) + ',' + ','.join([str(x) for x in results]) + '\n'
        fout.write(results)

--------- Batch id: 1 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.0
2022-02-23 00:00:00 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0]
--------- Batch id: 2 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.0
2022-02-23 00:00:00 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0]
--------- Batch id: 3 ---------


...

0.020284360989236576
2022-10-19 00:00:00 [10, 3, 0, 3, 0, 0, 11, 3, 3, 0]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 2] [6, 2, 6]
--------- Batch id: 111 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.02394927981992468
2022-10-19 00:00:00 [7, 3, 2, 5, 4, 8, 2, 4, 1, 3]
[3, 3, 3, 4, 4, 4, 4, 3, 3, 3] [4, 3, 0]
--------- Batch id: 112 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.024728244746578403
2022-10-19 00:00:00 [6, 2, 6, 5, 3, 4, 3, 2, 7, 2]
[3, 4, 3, 4, 4, 4, 4, 4, 4, 3] [0, 0, 0]
